In [29]:
import pandas as pd
from collections import defaultdict
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
from sklearn.preprocessing import normalize

In [4]:
# does not contain 'H' or 'AB' data
condensed_df = pd.read_csv('src/data/condensed.csv')

In [5]:
condensed_df.columns

Index(['playerID', 'year1_2B', 'year1_3B', 'year1_BB', 'year1_CS', 'year1_G',
       'year1_GIDP', 'year1_HBP', 'year1_HR', 'year1_IBB',
       ...
       'year7_SF', 'year7_SH', 'year7_SO', 'year7_avg', 'pos_1B', 'pos_2B',
       'pos_3B', 'pos_C', 'pos_OF', 'pos_SS'],
      dtype='object', length=119)

In [6]:
six_year_cols = [col for col in condensed_df.columns if 'year7' not in col]
six_year_df = condensed_df[six_year_cols]
y = condensed_df['year7_avg']

In [7]:
six_year_df2 = six_year_df.astype(float)

ValueError: could not convert string to float: 'zobribe01'

In [ ]:
uncondensed_df = pd.read_csv('src/data/uncondensed.csv')
uncondensed_df.drop('Unnamed: 0', axis=1, inplace=True)
uncondensed_df = pd.get_dummies(uncondensed_df, columns=['pos'])

In [ ]:
uncondensed_df.drop('yearID', axis=1, inplace=True)
uncondensed_df.set_index('playerID', inplace=True)

In [ ]:
uncondensed_df = uncondensed_df.astype(float)
uncondensed_df

In [ ]:
# X = uncondensed_df.groupby('playerID').apply(lambda x: x.iloc[:-1, :]).reset_index(0, drop=True)

# y = uncondensed_df.groupby('playerID').apply(lambda x: x.iloc[-1, :])

# y = pd.DataFrame(y['avg'])
# y

In [ ]:
y_values = uncondensed_df.pop('avg')

In [ ]:
X_panel = pd.Panel(dict_convert(uncondensed_df))
y_panel = pd.Panel(dict_convert(pd.DataFrame(y_values)))

In [ ]:
X_panel['year1'].T

In [ ]:
X_data = X_panel.as_matrix().astype('float32')
y_data = y_panel.as_matrix()
print(X_data.shape, y_data.shape)

In [ ]:
# scaler = MinMaxScaler(feature_range=(0, 2))
# data_norm = scaler.fit_transform(dataset.reshape(dataset.shape[0], -1)).reshape(dataset.shape)

In [ ]:
X_new_data = np.moveaxis(X_data, 0, 1)
y_new_data = np.moveaxis(y_data, 0, 1)
print(X_new_data.shape, y_new_data.shape)

In [ ]:
X_new_data[0, :, 0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new_data, y_new_data)

In [ ]:
y_test

In [ ]:
y_train = y_train[:,6,:] # grab only the last column of BAs
y_test = y_test[:,6,:]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(7, 21)))
model.add(Dense(7))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, verbose=1, epochs=5, batch_size=1)

In [ ]:
trainPred = model.predict(X_train)
testPred = model.predict(X_test)

print(trainPred.shape)
print(testPred.shape)

In [ ]:
print(y_test.shape)
print(testPred.shape)

In [ ]:
testScore = math.sqrt(mean_squared_error(y_test, testPred))
trainScore = math.sqrt(mean_squared_error(y_train, trainPred))
print('Test Score: {:.3f}'.format(testScore))
print('Train Score: {:.3f}'.format(trainScore))

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_train[0,1,:]

In [ ]:
y_train

# reincorporate BA
---

In [8]:
def dict_convert(df, length):
    def_dict = defaultdict(dict)
    if length == 1:
        return df.T.to_dict()
    for i in range(length):
        year = 'year' + str(i + 1)
        def_dict[year] = df.groupby('playerID').nth(i).T.to_dict('index')
    return def_dict

In [257]:
df = pd.read_csv('src/data/uncondensed.csv')
df.drop(['Unnamed: 0', 'yearID'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['pos'])
df.set_index('playerID', inplace=True, drop=True)
df

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,122,58,27,6,13,69.0,2.0,2.0,28,39.0,...,6.0,4.0,13.0,0.279915,0,0,0,0,1,0
aaronha01,153,105,37,9,27,106.0,3.0,1.0,49,61.0,...,7.0,4.0,20.0,0.313953,0,0,0,0,1,0
aaronha01,153,106,34,14,26,92.0,2.0,4.0,37,54.0,...,5.0,7.0,21.0,0.328407,0,0,0,0,1,0
aaronha01,151,118,27,6,44,132.0,1.0,1.0,57,58.0,...,0.0,3.0,13.0,0.321951,0,0,0,0,1,0
aaronha01,153,109,34,4,30,95.0,4.0,1.0,59,49.0,...,0.0,3.0,21.0,0.326123,0,0,0,0,1,0
aaronha01,154,116,46,7,39,123.0,8.0,0.0,51,54.0,...,0.0,9.0,19.0,0.354531,0,0,0,0,1,0
aaronha01,153,102,20,11,40,126.0,16.0,7.0,60,63.0,...,0.0,12.0,8.0,0.291525,0,0,0,0,1,0
abbotku01,101,41,17,3,9,33.0,3.0,0.0,16,98.0,...,3.0,2.0,5.0,0.249275,0,0,0,0,0,1
abbotku01,120,60,18,7,17,60.0,4.0,3.0,36,110.0,...,2.0,5.0,6.0,0.254762,0,0,0,0,0,1


In [282]:
X = df.groupby('playerID').head(6)

In [283]:
y = df['avg'].groupby('playerID').tail(6)
y = pd.DataFrame(y)

In [284]:
X

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,122,58,27,6,13,69.0,2.0,2.0,28,39.0,...,6.0,4.0,13.0,0.279915,0,0,0,0,1,0
aaronha01,153,105,37,9,27,106.0,3.0,1.0,49,61.0,...,7.0,4.0,20.0,0.313953,0,0,0,0,1,0
aaronha01,153,106,34,14,26,92.0,2.0,4.0,37,54.0,...,5.0,7.0,21.0,0.328407,0,0,0,0,1,0
aaronha01,151,118,27,6,44,132.0,1.0,1.0,57,58.0,...,0.0,3.0,13.0,0.321951,0,0,0,0,1,0
aaronha01,153,109,34,4,30,95.0,4.0,1.0,59,49.0,...,0.0,3.0,21.0,0.326123,0,0,0,0,1,0
aaronha01,154,116,46,7,39,123.0,8.0,0.0,51,54.0,...,0.0,9.0,19.0,0.354531,0,0,0,0,1,0
abbotku01,101,41,17,3,9,33.0,3.0,0.0,16,98.0,...,3.0,2.0,5.0,0.249275,0,0,0,0,0,1
abbotku01,120,60,18,7,17,60.0,4.0,3.0,36,110.0,...,2.0,5.0,6.0,0.254762,0,0,0,0,0,1
abbotku01,109,37,18,7,8,33.0,3.0,3.0,22,99.0,...,4.0,0.0,7.0,0.253125,0,0,0,0,0,1


In [285]:
from sklearn.preprocessing import Normalizer

In [320]:
def normalize_feats(df):
    cols = [col for col in X.columns if 'pos' not in col]
    cols.remove('avg')
    df[cols] = normalize(df[cols])
    return df

In [289]:
X

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,0.749638,0.356385,0.165904,0.036867,0.079879,0.423976,0.012289,0.012289,0.172048,0.239638,...,0.036867,0.024578,0.079879,0.279915,0,0,0,0,1,0
aaronha01,0.655500,0.449853,0.158520,0.038559,0.115677,0.454137,0.012853,0.004284,0.209931,0.261343,...,0.029990,0.017137,0.085686,0.313953,0,0,0,0,1,0
aaronha01,0.684257,0.474061,0.152057,0.062612,0.116279,0.411449,0.008945,0.017889,0.165474,0.241503,...,0.022361,0.031306,0.093918,0.328407,0,0,0,0,1,0
aaronha01,0.597496,0.466918,0.106837,0.023742,0.174105,0.522315,0.003957,0.003957,0.225545,0.229502,...,0.000000,0.011871,0.051440,0.321951,0,0,0,0,1,0
aaronha01,0.664633,0.473497,0.147696,0.017376,0.130320,0.412681,0.017376,0.004344,0.256297,0.212856,...,0.000000,0.013032,0.091224,0.326123,0,0,0,0,1,0
aaronha01,0.616914,0.464688,0.184273,0.028042,0.156231,0.492730,0.032047,0.000000,0.204303,0.216320,...,0.000000,0.036053,0.076113,0.354531,0,0,0,0,1,0
abbotku01,0.661915,0.268698,0.111411,0.019661,0.058982,0.216269,0.019661,0.000000,0.104858,0.642254,...,0.019661,0.013107,0.032768,0.249275,0,0,0,0,0,1
abbotku01,0.634317,0.317159,0.095148,0.037002,0.089862,0.317159,0.021144,0.015858,0.190295,0.581457,...,0.010572,0.026430,0.031716,0.254762,0,0,0,0,0,1
abbotku01,0.687263,0.233291,0.113493,0.044136,0.050441,0.208070,0.018915,0.018915,0.138714,0.624211,...,0.025221,0.000000,0.044136,0.253125,0,0,0,0,0,1


In [290]:
X_dict = dict_convert(X, 6)
y_dict = dict_convert(y, 6)

In [291]:
X_panel1 = pd.Panel(X_dict)
y_panel1 = pd.Panel(y_dict)

print(X_panel1.shape)
print(y_panel1.shape)

(6, 2348, 22)
(6, 2348, 1)


In [292]:
X_matrix = X_panel1.as_matrix().astype('float32')
y_matrix = y_panel1.as_matrix()
print(X_matrix.shape)
print(y_matrix.shape)

X_matrix = np.moveaxis(X_matrix, 0, 1)
y_matrix = np.moveaxis(y_matrix, 0, 1)
print(X_matrix.shape)
print(y_matrix.shape)

(6, 2348, 22)
(6, 2348, 1)
(2348, 6, 22)
(2348, 6, 1)


In [293]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix, y_matrix, test_size=0.2)
print('X_train:\t{}\n'\
      'X_test:\t\t{}\n'\
      'y_train:\t{}\n'\
      'y_test:\t\t{}\n'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

X_train:	(1878, 6, 22)
X_test:		(470, 6, 22)
y_train:	(1878, 6, 1)
y_test:		(470, 6, 1)



In [295]:
y_train1 = y_train[:,-1,:]
y_test1 = y_test[:,-1,:]
print('X_train:\t{}\n'\
      'X_test:\t\t{}\n'\
      'y_train1:\t{}\n'\
      'y_test1:\t\t{}\n'.format(X_train.shape, X_test.shape, y_train1.shape, y_test1.shape))

X_train:	(1878, 6, 22)
X_test:		(470, 6, 22)
y_train1:	(1878, 1)
y_test1:		(470, 1)



In [296]:
model1 = Sequential()
model1.add(LSTM(28, input_shape=(6, 22), return_sequences=True))
model1.add(BatchNormalization())
model1.add(Dropout(.3))
model1.add(LSTM(28))
model1.add(BatchNormalization())
model1.add(Dense(64))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.fit(X_train, y_train1, verbose=1, epochs=20, batch_size=10)

Epoch 1/20
1878/1878 [==============================] - 4s 2ms/step - loss: 0.0499
Epoch 2/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0024
Epoch 3/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0019
Epoch 4/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0020
Epoch 5/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0020
Epoch 6/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0023
Epoch 7/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0023
Epoch 8/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0025
Epoch 9/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0022
Epoch 10/20
1878/1878 [==============================] - 2s 1000us/step - loss: 0.0020
Epoch 11/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0027
Epoch 12/20
1878/1878 [==============================] - 2s 1ms/step - loss: 0.002

In [297]:
testPred = model1.predict(X_test)
print(testPred.shape)

(470, 1)


In [298]:
model1.evaluate(X_test, y_test1)

470/470 [==============================] - 1s 2ms/step


0.002407479950008874

In [299]:
X_test.shape

(470, 6, 22)

In [300]:
results = pd.DataFrame(y_test1)

In [301]:
results.columns = ['actual']

In [302]:
results['predicted'] = testPred

In [303]:
results = results.round(3)

In [304]:
results['difference'] = (results['actual'] - results['predicted']).abs()

In [305]:
results = results.round(3)

In [306]:
results

,actual,predicted,difference
0,0.328,0.236,0.092
1,0.278,0.238,0.040
2,0.269,0.320,0.051
3,0.312,0.243,0.069
4,0.209,0.284,0.075
5,0.334,0.302,0.032
6,0.310,0.236,0.074
7,0.234,0.224,0.010
8,0.317,0.276,0.041
9,0.266,0.275,0.009


In [307]:
results.sort_values('difference')

,actual,predicted,difference
193,0.308,0.308,0.000
330,0.227,0.227,0.000
391,0.283,0.283,0.000
58,0.232,0.232,0.000
144,0.251,0.251,0.000
80,0.232,0.232,0.000
306,0.257,0.257,0.000
367,0.272,0.272,0.000
345,0.251,0.251,0.000
17,0.258,0.258,0.000


In [308]:
print(results['difference'].max())

0.174


# predict one player 
---

In [367]:
avail_players = X.index

In [369]:
playerID = 'aaronha01'

In [370]:
if playerID in avail_players:
    X1 = X[X.index == playerID]
    y1 = y[y.index == playerID]
else:
    print('Not enough data')

In [371]:
X1

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,0.749638,0.356385,0.165904,0.036867,0.079879,0.423976,0.012289,0.012289,0.172048,0.239638,...,0.036867,0.024578,0.079879,0.279915,0,0,0,0,1,0
aaronha01,0.655500,0.449853,0.158520,0.038559,0.115677,0.454137,0.012853,0.004284,0.209931,0.261343,...,0.029990,0.017137,0.085686,0.313953,0,0,0,0,1,0
aaronha01,0.684257,0.474061,0.152057,0.062612,0.116279,0.411449,0.008945,0.017889,0.165474,0.241503,...,0.022361,0.031306,0.093918,0.328407,0,0,0,0,1,0
aaronha01,0.597496,0.466918,0.106837,0.023742,0.174105,0.522315,0.003957,0.003957,0.225545,0.229502,...,0.000000,0.011871,0.051440,0.321951,0,0,0,0,1,0
aaronha01,0.664633,0.473497,0.147696,0.017376,0.130320,0.412681,0.017376,0.004344,0.256297,0.212856,...,0.000000,0.013032,0.091224,0.326123,0,0,0,0,1,0
aaronha01,0.616914,0.464688,0.184273,0.028042,0.156231,0.492730,0.032047,0.000000,0.204303,0.216320,...,0.000000,0.036053,0.076113,0.354531,0,0,0,0,1,0


In [372]:
X_all = X[X.index != playerID]
y_all = y[y.index != playerID]

In [373]:
X_all_normed = normalize_feats(X_all)

/Users/aaronmcdonald/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/aaronmcdonald/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [374]:
X1_dict = dict_convert(X1, 6)
y1_dict = dict_convert(y1, 6)

X_all_dict = dict_convert(X_all_normed, 6)
y_all_dict = dict_convert(y_all, 6)

In [375]:
X1_panel = pd.Panel(X1_dict)
y1_panel = pd.Panel(y1_dict)

X_all_panel = pd.Panel(X_all_dict)
y_all_panel = pd.Panel(y_all_dict)

In [376]:
X1_matrix = X1_panel.as_matrix().astype('float32')
y1_matrix = y1_panel.as_matrix()
X1_matrix = np.moveaxis(X1_matrix, 0, 1)
y1_matrix = np.moveaxis(y1_matrix, 0, 1)
y1_matrix = y1_matrix[:,-1,:]

X_all_matrix = X_all_panel.as_matrix().astype('float32')
y_all_matrix = y_all_panel.as_matrix()
X_all_matrix = np.moveaxis(X_all_matrix, 0, 1)
y_all_matrix = np.moveaxis(y_all_matrix, 0, 1)
y_all_matrix = y_all_matrix[:,-1,:]

In [377]:
X_all_matrix.shape

(2347, 6, 22)

In [378]:
y_all_matrix.shape

(2347, 1)

In [379]:
def fit_lstm(X_train, y_train):
    lstm_model = Sequential()
    lstm_model.add(LSTM(28, input_shape=(6, 22), return_sequences=True))
    lstm_model.add(BatchNormalization())
    lstm_model.add(Dropout(.3))
    lstm_model.add(LSTM(28))
    lstm_model.add(BatchNormalization())
    lstm_model.add(Dense(64))
    lstm_model.add(Dense(1))
    lstm_model.compile(loss='mean_squared_error', optimizer='adam')
    lstm_model.fit(X_train, y_train, verbose=1, epochs=20, batch_size=10)
    
    return lstm_model

In [380]:
def display_results(model, X_test, y_test):
    y_pred = model.predict(X_test)
    df = pd.DataFrame(y_test)
    df.columns = ['actual']
    df['predicted'] = y_pred
    df = df.round(3)
    df['difference'] = (df['actual'] - df['predicted']).abs()
    return df

In [381]:
lstm_model = fit_lstm(X_all_matrix, y_all_matrix)

Epoch 1/20
2347/2347 [==============================] - 5s 2ms/step - loss: 0.0590
Epoch 2/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0031
Epoch 3/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0020
Epoch 4/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0019
Epoch 5/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0021
Epoch 6/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0026
Epoch 7/20
2347/2347 [==============================] - 2s 951us/step - loss: 0.0024
Epoch 8/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0024
Epoch 9/20
2347/2347 [==============================] - 3s 1ms/step - loss: 0.0024
Epoch 10/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0032
Epoch 11/20
2347/2347 [==============================] - 3s 1ms/step - loss: 0.0024
Epoch 12/20
2347/2347 [==============================] - 2s 1ms/step - loss: 0.0023

In [382]:
display_results(lstm_model, X1_matrix, y1_matrix)

,actual,predicted,difference
0,0.292,0.274,0.018
